In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Flatten
import pickle

2022-05-26 21:47:46.568574: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-26 21:47:46.568634: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
IoT = pd.read_csv('../../data/IoT_Garage_Door.csv')
IoT = IoT.iloc[0:500]

IoT = IoT.dropna()

encoder=LabelEncoder()
IoT['type']=encoder.fit_transform(IoT['type'])
IoT['state']=encoder.fit_transform(IoT['state'])
IoT['sphone_signal']=encoder.fit_transform(IoT['sphone_signal'])


IoT.head()

/tmp/ipykernel_24306/370847493.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  IoT = pd.read_csv('../../data/IoT_Garage_Door.csv')


,date,time,state,sphone_signal,label,type
0,1-Apr-19,20:53:44,1,1,0,0
1,1-Apr-19,20:53:49,0,0,0,0
2,1-Apr-19,20:53:49,1,1,0,0
3,1-Apr-19,20:53:54,0,0,0,0
4,1-Apr-19,20:53:54,1,1,0,0


In [3]:
x = IoT.drop(['label', 'date', 'time'], axis=1)
y = IoT['label']

features=len(x.columns)

x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)

In [4]:
model1 = Sequential()
    
model1.add(LSTM(units = 128, activation='tanh', return_sequences=True, input_shape = (features,1)))
model1.add(Dropout(0.2))
model1.add(LSTM(units = 64, activation='tanh', return_sequences=True))
model1.add(Dropout(0.2))
model1.add(Dense(1, activation='sigmoid')) 
model1.add(Flatten())
    
model1.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

2022-05-26 21:47:50.174337: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-26 21:47:50.174591: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-26 21:47:50.174668: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-05-26 21:47:50.174717: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-05-26 21:47:50.174766: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [5]:
model1.save('../h5s/LSTM.h5') 
# filename = '../h5s/LSTM.h5'
# pickle.dump(model1, open(filename, 'wb'))

In [6]:
model1.fit(x_train, np.asarray(y_train).astype('float32').reshape((-1,1)), epochs = 5, batch_size=32, validation_data=(x_test, np.asarray(y_test).astype('float32').reshape((-1,1))))

Epoch 1/5
13/13 [==============================] - 4s 68ms/step - loss: 0.5961 - accuracy: 0.9975 - val_loss: 0.4302 - val_accuracy: 1.0000
Epoch 2/5
13/13 [==============================] - 0s 10ms/step - loss: 0.3222 - accuracy: 1.0000 - val_loss: 0.2350 - val_accuracy: 1.0000
Epoch 3/5
13/13 [==============================] - 0s 9ms/step - loss: 0.1988 - accuracy: 1.0000 - val_loss: 0.1613 - val_accuracy: 1.0000
Epoch 4/5
13/13 [==============================] - 0s 9ms/step - loss: 0.1406 - accuracy: 1.0000 - val_loss: 0.1179 - val_accuracy: 1.0000
Epoch 5/5
13/13 [==============================] - 0s 9ms/step - loss: 0.1029 - accuracy: 1.0000 - val_loss: 0.0847 - val_accuracy: 1.0000


In [7]:
predictions = model1.predict(x_test)
y_pred = [round(x[0]) for x in predictions]
y_pred

4/4 [==============================] - 1s 2ms/step


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [8]:
for i in range(3):
	print(x_test.values[i].tolist(), '=>', y_pred[i], '( expected ', y_train.values[i], ')')

[0, 0, 0] => 0 ( expected  0 )
[0, 0, 0] => 0 ( expected  0 )
[0, 0, 0] => 0 ( expected  0 )


In [9]:
score, acc = model1.evaluate(x_test, y_test, batch_size=1)

print('Test score:', score)
print('Test accuracy:', acc)

100/100 [==============================] - 1s 1ms/step - loss: 0.0847 - accuracy: 1.0000
Test score: 0.08470947295427322
Test accuracy: 1.0
